In [ ]:
import pymc as pm
import numpy as np
from datetime import datetime
import yfinance as yf 
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az

np.random.seed(101)

start = datetime(2022, 11, 15)
end = datetime(2022, 12, 31)

market = yf.Ticker('SPY').history(start=start,end=end)
stock = yf.Ticker('AAPL').history(start=start,end=end)
risk_free_rate = yf.Ticker('^TNX').history(start=start,end=end)

daily_returns = pd.DataFrame()
daily_returns['market'] = market['Close'].pct_change(1)*100
daily_returns['stock'] = stock['Close'].pct_change(1)*100
daily_returns['riskfree'] = (1 + risk_free_rate['Close'])**(1/360)-1

daily_returns = daily_returns.ffill()
daily_returns = daily_returns.dropna()

y = daily_returns['stock'] - daily_returns['riskfree']
x = daily_returns['market'] - daily_returns['riskfree']

test_size = 10
x_train = x[:-test_size]
y_train = y[:-test_size]
x_test = x[-test_size:]
y_test = y[-test_size:]

model = pm.Model()
with model:
    alpha = pm.Normal('alpha',mu=0.02,sigma=0.10)
    beta = pm.Normal('beta',mu=1.2,sigma=0.15)
    residual = pm.HalfStudentT('residual',sigma=0.20,nu=6)
    feature = pm.MutableData('feature',x_train,dims='feature_data')
    target = pm.MutableData('target',y_train,dims='target_data')
    target_expected = pm.Deterministic('target_expected',alpha + beta * feature, dims='feature_data')
    target_likelihood = pm.StudentT('target_likelihood', mu=target_expected,sigma=residual,nu=6,observed=target,dims='target_data')
    
idata = pm.sample_prior_predictive(samples=1000,model=model,return_inferencedata=True,random_seed=28)

az.plot_trace(idata.prior,kind='trace',var_names=['alpha','beta','residual'],legend=True)

az.plot_posterior(idata.prior,var_names=['alpha','beta','residual'],round_to=2)

